In [173]:
import numpy as np 
import torch 
import json 
# from my_utils import *
from sklearn.preprocessing import MinMaxScaler

In [174]:
import numpy as np 

scr_to_enc = {
    'U' : 0,
    'U2' : 1, 
    'U\'' : 2, 
    'F' : 3,
    'F2' : 4,
    'F\'' : 5,
    'D' : 6,
    'D2' : 7,
    'D\'' : 8,
    'L' : 9,
    'L2' : 10, 
    'L\'' : 11,
    'R' : 12,
    'R2' : 13, 
    'R\'' : 14, 
    'B' : 15,
    'B2' : 16, 
    'B\'' : 17,
    'fill' : 18
}

def find_all_sessions(data, exclude_sessions):
    run = True 
    i = 1

    list_of_sessions = []
    while (run):
        if (i not in exclude_sessions):

            session = f'session{i}'
            try:
                data[session]
            except:
                break
            
            list_of_sessions.append(f'session{i}')        
        i += 1
    return list_of_sessions

def getSolveTime(solve):
    return solve[0][1]

def getSolveScramble(solve):
    # print(solve)
    # scramble = solve[1]
    # print(len(scramble))
    return solve[1] # to be implemented

def getScrambleAndTime(data, session):
    session = data[session]
    length = len(session)

    # scrambles = np.array(list(map(getSolveScramble, session)))
    scrambles = list(map(getSolveScramble, session))
    # print(scrambles)
    scrambles = list(map(convertStringToEncoding, scrambles))
    # print(scrambles)
    scrambles = np.array(scrambles)
    times = np.array(list(map(getSolveTime, session))) / 1000

    assert(len(scrambles) == len(times))
    return scrambles, times

def convertStringToEncoding(scramble):
    scramble = scramble.split(' ')
    enc = []
    for move in scramble:
        enc.append(scr_to_enc[move])
    # print(len(enc))

    while (len(enc) < 21):
        enc.append(scr_to_enc['fill'])

    return enc 

def generate_data(data, sessions):
    data_points = []

    all_scrambles = None
    all_times = None 

    for session in sessions: 
        scrambles, times = getScrambleAndTime(data, session)
        # print(scrambles)
        # print(times)
        # print(type(scrambles), type(times))
        # break 

        if all_scrambles is None:
            all_scrambles = scrambles 
            all_times = times 
            # print(all_scrambles.shape)
            # print(all_times.shape)
            # break
        else:
            all_scrambles = np.vstack((all_scrambles, scrambles))
            all_times = np.hstack((all_times, times))

    # print(all_scrambles.shape)
    # print(all_times.shape)

    data_points = np.array(data_points)
    return all_scrambles, all_times 

def generate_data_pairs(scrambles, times):
    num_solves, _ = scrambles.shape
    paired_data = np.hstack((scrambles, times.reshape(-1, 1)))

    return paired_data


In [175]:
f = open('cstimer_20230123_120616.txt')
data = json.load(f)

In [176]:
exclude_sessions = [1]
sessions = find_all_sessions(data, exclude_sessions)

In [180]:
scrambles, times = generate_data(data, sessions)
paired_data = generate_data_pairs(scrambles, times)
scaler = MinMaxScaler()
scaler.fit(paired_data)
scaled_data = scaler.transform(paired_data)


In [181]:
import torch
import torchvision
import torchvision.transforms as transforms


In [182]:
batch_size = 256
dataloader = torch.utils.data.DataLoader(scaled_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


In [204]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.network = nn.Sequential(
            nn.Linear(21, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.network(x)
        return x


net = Net()


In [205]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print("Device:", DEVICE)

Device: cuda


In [206]:
net = net.to(DEVICE)

In [207]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [209]:
epochs = 10
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch_data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
        scrambles = batch_data[:, :21]
        times = batch_data[:, 21]
        scrambles = scrambles.to(DEVICE)
        times = times.to(DEVICE)
        # print(times.device, scrambles.device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(scrambles.float())
        # print(outputs.device)
        loss = criterion(outputs, times.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0

print('Finished Training')


Finished Training
